In [1]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 14.1 MB/s eta 0:00:00


# optuna를 사용하여 AutoML 파이프라인을 코드로 구현

# 구현(끼워맞추기)

In [133]:
from typing import Dict, Any, List
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim
import optuna

In [134]:
def search_hyperparam(trial: optuna.trial.Trial) -> Dict[str, Any]:
    """Optuna를 사용하여 하이퍼파라미터를 탐색합니다."""
    return {
        "lr": trial.suggest_loguniform("lr", 1e-5, 1e-1),
        "optimizer": trial.suggest_categorical("optimizer", ["Adam", "SGD", "RMSprop"]),
        "batch_size": trial.suggest_categorical("batch_size", [16, 32, 64])
    }

In [135]:
def search_model(trial: optuna.trial.Trial) -> nn.Module:
    """Optuna를 사용하여 모델 구조를 탐색합니다."""
    hidden_size = trial.suggest_int("hidden_size", 32, 256)
    model = nn.Sequential(
        nn.Linear(10, hidden_size),
        nn.ReLU(),
        nn.Linear(hidden_size, 2)
    )
    return model

In [136]:
def train_model(
	model: nn.Module,
	hyperparams:Dict[str, Any]
	) -> nn.Module:
  pass

In [137]:
# search_hyperparam 과 search_model 함수를 사용하여 주어진 탐색 공간에서 모델과 하이퍼파라미터를 가져옴

In [138]:
def train_model(model: nn.Module, hyperparams: Dict[str, Any], train_loader: DataLoader, epochs: int = 10) -> nn.Module:
    """모델을 학습하는 함수"""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = getattr(optim, hyperparams["optimizer"])(model.parameters(), lr=hyperparams["lr"])
    model.train()
    for epoch in range(epochs):
        total_loss = 0.0
        for batch in train_loader:
            inputs, targets = batch
            inputs, targets = inputs.to(device), targets.to(device)

            targets = targets.squeeze()  # (batch_size, 1) → (batch_size,)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}")
    return model

In [139]:
# 주어진 모델과 하이퍼파라미터를 사용하여 모델을 학습

In [140]:
def evaluate(model: nn.Module) -> float:
    """학습된 모델을 평가하는 함수"""
    return torch.rand(1).item()

In [141]:
# 학습된 모델의 성능을 측정

In [142]:
def objective(trial: optuna.trial.Trial) -> float:
  architecture = sample_model(trial)
  hyperparams = sample_hyperparam(trial)
  model = train_model(Model(architecture, verbose=True), hyperparams)
  score = evaluate(model)
  return score

In [68]:
if __name__ == "__main__":
  study = optuna.create_study(direction="maximize")
  study.optimize(objective, n_trials=500)
  print(f"Best trial {study.best_trial}")

In [69]:
# Optuna가 결과를 가지고 다음 모델과 하이퍼파라미터를 탐색

# 구현 실제

In [143]:
def objective(trial: optuna.trial.Trial) -> float:
    """Optuna 최적화 목표 함수"""
    hyperparams = search_hyperparam(trial)  # trial을 올바르게 전달
    model = search_model(trial)  # trial을 올바르게 전달
    train_loader = DataLoader([(torch.rand(10), torch.randint(0, 2, (1,))) for _ in range(100)], batch_size=hyperparams["batch_size"], shuffle=True)
    trained_model = train_model(model, hyperparams, train_loader)
    score = evaluate(trained_model)
    return score

In [96]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)

[I 2025-04-06 23:57:25,285] A new study created in memory with name: no-name-d9aa3f87-dae0-4358-97e9-89ff78fe74d4
<ipython-input-86-f21c17046f4f>:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "lr": trial.suggest_loguniform("lr", 1e-5, 1e-1),


Epoch 1, Loss: 0.7059
Epoch 2, Loss: 0.6908
Epoch 3, Loss: 0.6934
Epoch 4, Loss: 0.6901
Epoch 5, Loss: 0.6995
Epoch 6, Loss: 0.6889
Epoch 7, Loss: 0.6971
Epoch 8, Loss: 0.6852
Epoch 9, Loss: 0.6808
Epoch 10, Loss: 0.6804


[I 2025-04-06 23:57:33,163] Trial 0 finished with value: 0.6964746713638306 and parameters: {'lr': 0.025327373197835318, 'optimizer': 'SGD', 'batch_size': 32, 'hidden_size': 63}. Best is trial 0 with value: 0.6964746713638306.
<ipython-input-86-f21c17046f4f>:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "lr": trial.suggest_loguniform("lr", 1e-5, 1e-1),
[I 2025-04-06 23:57:33,335] Trial 1 finished with value: 0.06911718845367432 and parameters: {'lr': 3.205780775929534e-05, 'optimizer': 'SGD', 'batch_size': 16, 'hidden_size': 215}. Best is trial 0 with value: 0.6964746713638306.
[I 2025-04-06 23:57:33,398] Trial 2 finished with value: 0.6448426842689514 and parameters: {'lr': 0.015430158279002454, 'optimizer': 'RMSprop', 'batch_size': 64, 'hidden_size': 63}. Best is trial 0 with value: 0.6964746713638306.


Epoch 1, Loss: 0.6836
Epoch 2, Loss: 0.6883
Epoch 3, Loss: 0.6825
Epoch 4, Loss: 0.6948
Epoch 5, Loss: 0.6854
Epoch 6, Loss: 0.6806
Epoch 7, Loss: 0.6891
Epoch 8, Loss: 0.6870
Epoch 9, Loss: 0.6875
Epoch 10, Loss: 0.6850
Epoch 1, Loss: 2.7810
Epoch 2, Loss: 1.5705
Epoch 3, Loss: 0.6989
Epoch 4, Loss: 0.6671
Epoch 5, Loss: 0.6551
Epoch 6, Loss: 0.6567
Epoch 7, Loss: 0.6525
Epoch 8, Loss: 0.6429
Epoch 9, Loss: 0.6468
Epoch 10, Loss: 0.6319


[I 2025-04-06 23:57:33,437] Trial 3 finished with value: 0.4704211950302124 and parameters: {'lr': 9.415375578762937e-05, 'optimizer': 'Adam', 'batch_size': 64, 'hidden_size': 83}. Best is trial 0 with value: 0.6964746713638306.
[I 2025-04-06 23:57:33,524] Trial 4 finished with value: 0.21514463424682617 and parameters: {'lr': 0.00030678801519633356, 'optimizer': 'Adam', 'batch_size': 16, 'hidden_size': 106}. Best is trial 0 with value: 0.6964746713638306.
[I 2025-04-06 23:57:33,549] Trial 5 finished with value: 0.8719936609268188 and parameters: {'lr': 0.000959440783727542, 'optimizer': 'SGD', 'batch_size': 64, 'hidden_size': 187}. Best is trial 5 with value: 0.8719936609268188.
[I 2025-04-06 23:57:33,572] Trial 6 finished with value: 0.8683130741119385 and parameters: {'lr': 0.047064511559580566, 'optimizer': 'SGD', 'batch_size': 64, 'hidden_size': 172}. Best is trial 5 with value: 0.8719936609268188.
[I 2025-04-06 23:57:33,603] Trial 7 finished with value: 0.580981969833374 and para

Epoch 1, Loss: 0.6879
Epoch 2, Loss: 0.6885
Epoch 3, Loss: 0.6863
Epoch 4, Loss: 0.6878
Epoch 5, Loss: 0.6874
Epoch 6, Loss: 0.6895
Epoch 7, Loss: 0.6854
Epoch 8, Loss: 0.6853
Epoch 9, Loss: 0.6878
Epoch 10, Loss: 0.6879
Epoch 1, Loss: 0.6878
Epoch 2, Loss: 0.6838
Epoch 3, Loss: 0.6878
Epoch 4, Loss: 0.6871
Epoch 5, Loss: 0.6908
Epoch 6, Loss: 0.6798
Epoch 7, Loss: 0.6932
Epoch 8, Loss: 0.6893
Epoch 9, Loss: 0.6852
Epoch 10, Loss: 0.6805
Epoch 1, Loss: 0.7010
Epoch 2, Loss: 0.6963
Epoch 3, Loss: 0.7025
Epoch 4, Loss: 0.7003
Epoch 5, Loss: 0.6968
Epoch 6, Loss: 0.7004
Epoch 7, Loss: 0.7018
Epoch 8, Loss: 0.6969
Epoch 9, Loss: 0.7019
Epoch 10, Loss: 0.6998
Epoch 1, Loss: 0.6767
Epoch 2, Loss: 0.6766
Epoch 3, Loss: 0.6827
Epoch 4, Loss: 0.6749
Epoch 5, Loss: 0.6779
Epoch 6, Loss: 0.6764
Epoch 7, Loss: 0.6829
Epoch 8, Loss: 0.6765
Epoch 9, Loss: 0.6716
Epoch 10, Loss: 0.6783
Epoch 1, Loss: 2.2844
Epoch 2, Loss: 0.7864
Epoch 3, Loss: 0.8298
Epoch 4, Loss: 0.6852
Epoch 5, Loss: 0.7253
Epoch 

[I 2025-04-06 23:57:33,642] Trial 8 finished with value: 0.9106083512306213 and parameters: {'lr': 0.0002519596275309123, 'optimizer': 'SGD', 'batch_size': 32, 'hidden_size': 42}. Best is trial 8 with value: 0.9106083512306213.
[I 2025-04-06 23:57:33,665] Trial 9 finished with value: 0.6072807908058167 and parameters: {'lr': 0.0020793244846207723, 'optimizer': 'SGD', 'batch_size': 64, 'hidden_size': 91}. Best is trial 8 with value: 0.9106083512306213.


Epoch 3, Loss: 0.6984
Epoch 4, Loss: 0.6906
Epoch 5, Loss: 0.7101
Epoch 6, Loss: 0.7093
Epoch 7, Loss: 0.6928
Epoch 8, Loss: 0.7136
Epoch 9, Loss: 0.7240
Epoch 10, Loss: 0.6980
Epoch 1, Loss: 0.6962
Epoch 2, Loss: 0.6964
Epoch 3, Loss: 0.6973
Epoch 4, Loss: 0.6976
Epoch 5, Loss: 0.7016
Epoch 6, Loss: 0.6974
Epoch 7, Loss: 0.6962
Epoch 8, Loss: 0.6943
Epoch 9, Loss: 0.6957
Epoch 10, Loss: 0.6990


In [97]:
# 학습된 모델의 성능을 측정한 이후
# Optuna가 측정된 성능을 기반으로 다음 모델과 하이퍼파라미터를 탐색

In [98]:
print(f"Best trial: {study.best_trial}")

Best trial: FrozenTrial(number=8, state=1, values=[0.9106083512306213], datetime_start=datetime.datetime(2025, 4, 6, 23, 57, 33, 604361), datetime_complete=datetime.datetime(2025, 4, 6, 23, 57, 33, 642059), params={'lr': 0.0002519596275309123, 'optimizer': 'SGD', 'batch_size': 32, 'hidden_size': 42}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'lr': FloatDistribution(high=0.1, log=True, low=1e-05, step=None), 'optimizer': CategoricalDistribution(choices=('Adam', 'SGD', 'RMSprop')), 'batch_size': CategoricalDistribution(choices=(16, 32, 64)), 'hidden_size': IntDistribution(high=256, log=False, low=32, step=1)}, trial_id=8, value=None)


In [99]:
if __name__ == "__main__":

# 1. Optuna Study 생성 (blackbox Optimizer 및 관리 담당)
  study = optuna.create_study(direction='maximize')

# 2. Study에 최적화할 목적함수 및 시도 횟수, 조건 등을 주고 Optimize
  study.optimize(objective, n_trials=500)
  print(f"Best trial {study.best_trial}")

[I 2025-04-06 23:57:38,071] A new study created in memory with name: no-name-804f0d92-bbc5-454f-a1e8-965b10aa8ffb
<ipython-input-86-f21c17046f4f>:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "lr": trial.suggest_loguniform("lr", 1e-5, 1e-1),
[I 2025-04-06 23:57:38,101] Trial 0 finished with value: 0.943057656288147 and parameters: {'lr': 4.484684488187017e-05, 'optimizer': 'SGD', 'batch_size': 64, 'hidden_size': 50}. Best is trial 0 with value: 0.943057656288147.
[I 2025-04-06 23:57:38,196] Trial 1 finished with value: 0.8618950247764587 and parameters: {'lr': 0.013398312478068218, 'optimizer': 'Adam', 'batch_size': 16, 'hidden_size': 112}. Best is trial 0 with value: 0.943057656288147.
[I 2025-04-06 23:57:38,221] Trial 2 finished with value: 0.9549549221992493 and parameters: {'lr': 0.02005961005136102, 'optimizer': 'SGD', 

Epoch 1, Loss: 0.7499
Epoch 2, Loss: 0.7485
Epoch 3, Loss: 0.7484
Epoch 4, Loss: 0.7567
Epoch 5, Loss: 0.7514
Epoch 6, Loss: 0.7500
Epoch 7, Loss: 0.7502
Epoch 8, Loss: 0.7434
Epoch 9, Loss: 0.7524
Epoch 10, Loss: 0.7439
Epoch 1, Loss: 0.7348
Epoch 2, Loss: 0.7293
Epoch 3, Loss: 0.6890
Epoch 4, Loss: 0.6575
Epoch 5, Loss: 0.6730
Epoch 6, Loss: 0.6241
Epoch 7, Loss: 0.6437
Epoch 8, Loss: 0.6121
Epoch 9, Loss: 0.5946
Epoch 10, Loss: 0.5929
Epoch 1, Loss: 0.6929
Epoch 2, Loss: 0.6936
Epoch 3, Loss: 0.6923
Epoch 4, Loss: 0.6887
Epoch 5, Loss: 0.6850
Epoch 6, Loss: 0.6818
Epoch 7, Loss: 0.6914
Epoch 8, Loss: 0.6873
Epoch 9, Loss: 0.6863
Epoch 10, Loss: 0.6870
Epoch 1, Loss: 0.6892
Epoch 2, Loss: 0.6913
Epoch 3, Loss: 0.6919
Epoch 4, Loss: 0.6900
Epoch 5, Loss: 0.6922
Epoch 6, Loss: 0.6901
Epoch 7, Loss: 0.6927
Epoch 8, Loss: 0.6920
Epoch 9, Loss: 0.6921
Epoch 10, Loss: 0.6891
Epoch 1, Loss: 0.6948
Epoch 2, Loss: 0.7036
Epoch 3, Loss: 0.6707
Epoch 4, Loss: 0.6823
Epoch 5, Loss: 0.6765
Epoch 

[I 2025-04-06 23:57:38,299] Trial 4 finished with value: 0.2755178213119507 and parameters: {'lr': 0.0009677393692637627, 'optimizer': 'RMSprop', 'batch_size': 32, 'hidden_size': 56}. Best is trial 2 with value: 0.9549549221992493.
[I 2025-04-06 23:57:38,329] Trial 5 finished with value: 0.16908729076385498 and parameters: {'lr': 3.109241781030562e-05, 'optimizer': 'Adam', 'batch_size': 64, 'hidden_size': 185}. Best is trial 2 with value: 0.9549549221992493.
[I 2025-04-06 23:57:38,421] Trial 6 finished with value: 0.7039886116981506 and parameters: {'lr': 0.0008282944824458152, 'optimizer': 'Adam', 'batch_size': 16, 'hidden_size': 192}. Best is trial 2 with value: 0.9549549221992493.
[I 2025-04-06 23:57:38,473] Trial 7 finished with value: 0.34522950649261475 and parameters: {'lr': 5.255256152625119e-05, 'optimizer': 'Adam', 'batch_size': 32, 'hidden_size': 153}. Best is trial 2 with value: 0.9549549221992493.


Epoch 8, Loss: 0.6649
Epoch 9, Loss: 0.6676
Epoch 10, Loss: 0.6559
Epoch 1, Loss: 0.6956
Epoch 2, Loss: 0.6949
Epoch 3, Loss: 0.6986
Epoch 4, Loss: 0.6964
Epoch 5, Loss: 0.6957
Epoch 6, Loss: 0.6975
Epoch 7, Loss: 0.6968
Epoch 8, Loss: 0.6941
Epoch 9, Loss: 0.6934
Epoch 10, Loss: 0.6955
Epoch 1, Loss: 0.6924
Epoch 2, Loss: 0.6877
Epoch 3, Loss: 0.6827
Epoch 4, Loss: 0.6944
Epoch 5, Loss: 0.6749
Epoch 6, Loss: 0.6735
Epoch 7, Loss: 0.6717
Epoch 8, Loss: 0.6645
Epoch 9, Loss: 0.6616
Epoch 10, Loss: 0.6638
Epoch 1, Loss: 0.7354
Epoch 2, Loss: 0.7394
Epoch 3, Loss: 0.7141
Epoch 4, Loss: 0.7299
Epoch 5, Loss: 0.7493
Epoch 6, Loss: 0.7066
Epoch 7, Loss: 0.7045
Epoch 8, Loss: 0.7198
Epoch 9, Loss: 0.6933
Epoch 10, Loss: 0.7083
Epoch 1, Loss: 0.7170


[I 2025-04-06 23:57:38,525] Trial 8 finished with value: 0.256649374961853 and parameters: {'lr': 0.0015047030845101955, 'optimizer': 'RMSprop', 'batch_size': 32, 'hidden_size': 155}. Best is trial 2 with value: 0.9549549221992493.
[I 2025-04-06 23:57:38,563] Trial 9 finished with value: 0.6254755854606628 and parameters: {'lr': 0.0012556287059369267, 'optimizer': 'SGD', 'batch_size': 32, 'hidden_size': 131}. Best is trial 2 with value: 0.9549549221992493.
[I 2025-04-06 23:57:38,624] Trial 10 finished with value: 0.5937179327011108 and parameters: {'lr': 0.05976924050262233, 'optimizer': 'SGD', 'batch_size': 64, 'hidden_size': 244}. Best is trial 2 with value: 0.9549549221992493.
[I 2025-04-06 23:57:38,663] Trial 11 finished with value: 0.5961800813674927 and parameters: {'lr': 0.0001257709324155411, 'optimizer': 'SGD', 'batch_size': 64, 'hidden_size': 35}. Best is trial 2 with value: 0.9549549221992493.


...

Epoch 4, Loss: 0.7115
Epoch 5, Loss: 0.7038
Epoch 6, Loss: 0.7094
Epoch 7, Loss: 0.7040
Epoch 8, Loss: 0.6968
Epoch 9, Loss: 0.7083
Epoch 10, Loss: 0.7056
Epoch 1, Loss: 0.6946
Epoch 2, Loss: 0.6917
Epoch 3, Loss: 0.6850
Epoch 4, Loss: 0.6918
Epoch 5, Loss: 0.6874
Epoch 6, Loss: 0.6879
Epoch 7, Loss: 0.6931
Epoch 8, Loss: 0.6917
Epoch 9, Loss: 0.6901
Epoch 10, Loss: 0.6952
Epoch 1, Loss: 0.6999
Epoch 2, Loss: 0.7028
Epoch 3, Loss: 0.6996
Epoch 4, Loss: 0.6987
Epoch 5, Loss: 0.7005
Epoch 6, Loss: 0.7003
Epoch 7, Loss: 0.6996
Epoch 8, Loss: 0.7021
Epoch 9, Loss: 0.7024
Epoch 10, Loss: 0.7022


[I 2025-04-06 23:58:12,740] Trial 495 finished with value: 0.3427974581718445 and parameters: {'lr': 0.001621998335593861, 'optimizer': 'SGD', 'batch_size': 16, 'hidden_size': 174}. Best is trial 323 with value: 0.9971712827682495.
[I 2025-04-06 23:58:12,806] Trial 496 finished with value: 0.7219321131706238 and parameters: {'lr': 0.0012713362362430053, 'optimizer': 'SGD', 'batch_size': 64, 'hidden_size': 181}. Best is trial 323 with value: 0.9971712827682495.
[I 2025-04-06 23:58:12,869] Trial 497 finished with value: 0.546387255191803 and parameters: {'lr': 0.002541939964499509, 'optimizer': 'SGD', 'batch_size': 64, 'hidden_size': 188}. Best is trial 323 with value: 0.9971712827682495.


Epoch 1, Loss: 0.7003
Epoch 2, Loss: 0.7010
Epoch 3, Loss: 0.6998
Epoch 4, Loss: 0.7000
Epoch 5, Loss: 0.6993
Epoch 6, Loss: 0.6989
Epoch 7, Loss: 0.6989
Epoch 8, Loss: 0.6925
Epoch 9, Loss: 0.6952
Epoch 10, Loss: 0.6943
Epoch 1, Loss: 0.6899
Epoch 2, Loss: 0.6907
Epoch 3, Loss: 0.6904
Epoch 4, Loss: 0.6875
Epoch 5, Loss: 0.6873
Epoch 6, Loss: 0.6880
Epoch 7, Loss: 0.6888
Epoch 8, Loss: 0.6908
Epoch 9, Loss: 0.6884
Epoch 10, Loss: 0.6906
Epoch 1, Loss: 0.7013
Epoch 2, Loss: 0.6992
Epoch 3, Loss: 0.6969
Epoch 4, Loss: 0.7017
Epoch 5, Loss: 0.6967
Epoch 6, Loss: 0.7015
Epoch 7, Loss: 0.7015
Epoch 8, Loss: 0.6947
Epoch 9, Loss: 0.6954
Epoch 10, Loss: 0.6921


[I 2025-04-06 23:58:12,992] Trial 498 finished with value: 0.6471257209777832 and parameters: {'lr': 0.00017665696541684244, 'optimizer': 'Adam', 'batch_size': 16, 'hidden_size': 172}. Best is trial 323 with value: 0.9971712827682495.
[I 2025-04-06 23:58:13,061] Trial 499 finished with value: 0.42713457345962524 and parameters: {'lr': 0.00032275694218563046, 'optimizer': 'SGD', 'batch_size': 64, 'hidden_size': 170}. Best is trial 323 with value: 0.9971712827682495.


Epoch 1, Loss: 0.7188
Epoch 2, Loss: 0.7031
Epoch 3, Loss: 0.6917
Epoch 4, Loss: 0.6891
Epoch 5, Loss: 0.6854
Epoch 6, Loss: 0.6835
Epoch 7, Loss: 0.6810
Epoch 8, Loss: 0.6781
Epoch 9, Loss: 0.6677
Epoch 10, Loss: 0.6763
Epoch 1, Loss: 0.6940
Epoch 2, Loss: 0.6926
Epoch 3, Loss: 0.6894
Epoch 4, Loss: 0.6932
Epoch 5, Loss: 0.6898
Epoch 6, Loss: 0.6919
Epoch 7, Loss: 0.6944
Epoch 8, Loss: 0.6932
Epoch 9, Loss: 0.6929
Epoch 10, Loss: 0.6968
Best trial FrozenTrial(number=323, state=1, values=[0.9971712827682495], datetime_start=datetime.datetime(2025, 4, 6, 23, 57, 58, 925743), datetime_complete=datetime.datetime(2025, 4, 6, 23, 57, 58, 987553), params={'lr': 0.00017933278674080134, 'optimizer': 'SGD', 'batch_size': 64, 'hidden_size': 214}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'lr': FloatDistribution(high=0.1, log=True, low=1e-05, step=None), 'optimizer': CategoricalDistribution(choices=('Adam', 'SGD', 'RMSprop')), 'batch_size': CategoricalDistribution(cho

# 아키텍처 구성

In [150]:
def search_hyperparam(trial: optuna.trial.Trial):
    """Optuna를 사용하여 하이퍼파라미터를 탐색합니다."""
    optimizer = trial.suggest_categorical("optimizer", ["Adam", "SGD", "Adagrad"])

    # 옵티마이저에 따른 다른 하이퍼파라미터 설정
    if optimizer == "Adam":
        lr = trial.suggest_float(name="lr", low=1e-5, high=1e-3)
        beta1 = trial.suggest_float(name="beta1", low=0.8, high=0.95)  # Beta1 파라미터
        beta2 = trial.suggest_float(name="beta2", low=0.9, high=0.9999)  # Beta2 파라미터
    elif optimizer == "SGD":
        lr = trial.suggest_float(name="lr", low=1e-6, high=1e-4)
        momentum = trial.suggest_float(name="momentum", low=0.0, high=0.95)  # Momentum 파라미터
    elif optimizer == "Adagrad":
        lr = trial.suggest_float(name="lr", low=1e-5, high=1e-2)
        epsilon = trial.suggest_float(name="epsilon", low=1e-8, high=1e-4)  # Epsilon 파라미터

    return {
        # Optuna에서 주요 타입 정의
        "lr": trial.suggest_float("learning_rate", low=1e-5, high=1e-2, log=True),
        "optimizer": trial.suggest_categorical("optimizer", ["Adam", "SGD", "Adagrad"]),
        "batch_size": trial.suggest_categorical("batch_size", [16, 32, 64]),
        "epochs": trial.suggest_int("epochs", 10, 30, step=10),
        "activation": trial.suggest_categorical(name="m2/activation", choices=["ReLU", "ReLU6", "Hardswish"]),
    }

In [151]:
# Optuna Study 최적화 실행
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)

# 최적의 trial 출력
print(f"Best trial: {study.best_trial}")

[I 2025-04-07 00:37:22,870] A new study created in memory with name: no-name-7b8eb022-1b07-44f4-8005-5c50a2b800cd
[I 2025-04-07 00:37:22,918] Trial 0 finished with value: 0.9140507578849792 and parameters: {'optimizer': 'Adam', 'lr': 0.00023158400880456584, 'beta1': 0.8752557048214611, 'beta2': 0.9650089159984441, 'learning_rate': 1.5324326052059918e-05, 'batch_size': 64, 'epochs': 30, 'm2/activation': 'ReLU', 'hidden_size': 228}. Best is trial 0 with value: 0.9140507578849792.
[I 2025-04-07 00:37:22,996] Trial 1 finished with value: 0.1770249605178833 and parameters: {'optimizer': 'SGD', 'lr': 4.268292014089848e-05, 'momentum': 0.3065221689611961, 'learning_rate': 0.0011175306447119171, 'batch_size': 16, 'epochs': 30, 'm2/activation': 'ReLU', 'hidden_size': 202}. Best is trial 0 with value: 0.9140507578849792.
[I 2025-04-07 00:37:23,024] Trial 2 finished with value: 0.34031397104263306 and parameters: {'optimizer': 'SGD', 'lr': 2.79474119760596e-05, 'momentum': 0.8301404250662792, 'le

Epoch 1, Loss: 0.6966
Epoch 2, Loss: 0.6977
Epoch 3, Loss: 0.6947
Epoch 4, Loss: 0.6946
Epoch 5, Loss: 0.6982
Epoch 6, Loss: 0.6950
Epoch 7, Loss: 0.6985
Epoch 8, Loss: 0.6968
Epoch 9, Loss: 0.6966
Epoch 10, Loss: 0.6962
Epoch 1, Loss: 0.6942
Epoch 2, Loss: 0.6974
Epoch 3, Loss: 0.6960
Epoch 4, Loss: 0.6966
Epoch 5, Loss: 0.6949
Epoch 6, Loss: 0.6942
Epoch 7, Loss: 0.6937
Epoch 8, Loss: 0.6974
Epoch 9, Loss: 0.6960
Epoch 10, Loss: 0.6937
Epoch 1, Loss: 0.7121
Epoch 2, Loss: 0.7127
Epoch 3, Loss: 0.7106
Epoch 4, Loss: 0.7113
Epoch 5, Loss: 0.7106
Epoch 6, Loss: 0.7109
Epoch 7, Loss: 0.7094
Epoch 8, Loss: 0.7115
Epoch 9, Loss: 0.7092
Epoch 10, Loss: 0.7159
Epoch 1, Loss: 0.6956
Epoch 2, Loss: 0.7013
Epoch 3, Loss: 0.7019
Epoch 4, Loss: 0.6971
Epoch 5, Loss: 0.6952
Epoch 6, Loss: 0.7001
Epoch 7, Loss: 0.6996
Epoch 8, Loss: 0.6976
Epoch 9, Loss: 0.6988
Epoch 10, Loss: 0.6963
Epoch 1, Loss: 0.7213
Epoch 2, Loss: 0.7118
Epoch 3, Loss: 0.7170
Epoch 4, Loss: 0.7070


[I 2025-04-07 00:37:23,124] Trial 4 finished with value: 0.25153446197509766 and parameters: {'optimizer': 'SGD', 'lr': 5.40689803447582e-05, 'momentum': 0.45306674988158413, 'learning_rate': 0.0020653604342663434, 'batch_size': 16, 'epochs': 30, 'm2/activation': 'Hardswish', 'hidden_size': 80}. Best is trial 0 with value: 0.9140507578849792.
[I 2025-04-07 00:37:23,178] Trial 5 finished with value: 0.27559882402420044 and parameters: {'optimizer': 'SGD', 'lr': 6.026193400348613e-05, 'momentum': 0.5841804610004194, 'learning_rate': 1.3231334912466074e-05, 'batch_size': 32, 'epochs': 30, 'm2/activation': 'ReLU', 'hidden_size': 122}. Best is trial 0 with value: 0.9140507578849792.
[I 2025-04-07 00:37:23,267] Trial 6 finished with value: 0.3727840185165405 and parameters: {'optimizer': 'Adam', 'lr': 0.00016668227360107916, 'beta1': 0.9227344336800404, 'beta2': 0.9778135922219043, 'learning_rate': 0.0035675518725370695, 'batch_size': 16, 'epochs': 30, 'm2/activation': 'ReLU', 'hidden_size':

Epoch 5, Loss: 0.7115
Epoch 6, Loss: 0.7125
Epoch 7, Loss: 0.7092
Epoch 8, Loss: 0.7171
Epoch 9, Loss: 0.7017
Epoch 10, Loss: 0.6974
Epoch 1, Loss: 0.6970
Epoch 2, Loss: 0.6827
Epoch 3, Loss: 0.6846
Epoch 4, Loss: 0.6969
Epoch 5, Loss: 0.6866
Epoch 6, Loss: 0.6817
Epoch 7, Loss: 0.6887
Epoch 8, Loss: 0.6886
Epoch 9, Loss: 0.6910
Epoch 10, Loss: 0.6857
Epoch 1, Loss: 0.7072
Epoch 2, Loss: 0.6759
Epoch 3, Loss: 0.6663
Epoch 4, Loss: 0.6607
Epoch 5, Loss: 0.6349
Epoch 6, Loss: 0.6390
Epoch 7, Loss: 0.6138
Epoch 8, Loss: 0.5962
Epoch 9, Loss: 0.6036
Epoch 10, Loss: 0.6085
Epoch 1, Loss: 0.6949
Epoch 2, Loss: 0.7007
Epoch 3, Loss: 0.7004
Epoch 4, Loss: 0.6990
Epoch 5, Loss: 0.6923


[I 2025-04-07 00:37:23,315] Trial 7 finished with value: 0.8390475511550903 and parameters: {'optimizer': 'SGD', 'lr': 8.91835439369e-06, 'momentum': 0.648259428948649, 'learning_rate': 1.024931349408502e-05, 'batch_size': 32, 'epochs': 10, 'm2/activation': 'ReLU', 'hidden_size': 107}. Best is trial 0 with value: 0.9140507578849792.
[I 2025-04-07 00:37:23,368] Trial 8 finished with value: 0.3425139784812927 and parameters: {'optimizer': 'Adam', 'lr': 0.0004125173763145874, 'beta1': 0.9392859167948535, 'beta2': 0.9774901062224244, 'learning_rate': 0.0010575822873309052, 'batch_size': 32, 'epochs': 10, 'm2/activation': 'Hardswish', 'hidden_size': 106}. Best is trial 0 with value: 0.9140507578849792.
[I 2025-04-07 00:37:23,407] Trial 9 finished with value: 0.13355612754821777 and parameters: {'optimizer': 'SGD', 'lr': 1.1445831298852527e-05, 'momentum': 0.5569532185592495, 'learning_rate': 0.0007227885067267652, 'batch_size': 32, 'epochs': 30, 'm2/activation': 'Hardswish', 'hidden_size': 

Epoch 6, Loss: 0.6948
Epoch 7, Loss: 0.6958
Epoch 8, Loss: 0.7026
Epoch 9, Loss: 0.7019
Epoch 10, Loss: 0.6995
Epoch 1, Loss: 0.6982
Epoch 2, Loss: 0.6978
Epoch 3, Loss: 0.6871
Epoch 4, Loss: 0.6917
Epoch 5, Loss: 0.6903
Epoch 6, Loss: 0.6811
Epoch 7, Loss: 0.6851
Epoch 8, Loss: 0.6923
Epoch 9, Loss: 0.6825
Epoch 10, Loss: 0.6797
Epoch 1, Loss: 0.7183
Epoch 2, Loss: 0.7124
Epoch 3, Loss: 0.7241
Epoch 4, Loss: 0.7095
Epoch 5, Loss: 0.7060
Epoch 6, Loss: 0.7017
Epoch 7, Loss: 0.7030
Epoch 8, Loss: 0.6999
Epoch 9, Loss: 0.7022
Epoch 10, Loss: 0.6952
Best trial: FrozenTrial(number=0, state=1, values=[0.9140507578849792], datetime_start=datetime.datetime(2025, 4, 7, 0, 37, 22, 872428), datetime_complete=datetime.datetime(2025, 4, 7, 0, 37, 22, 917971), params={'optimizer': 'Adam', 'lr': 0.00023158400880456584, 'beta1': 0.8752557048214611, 'beta2': 0.9650089159984441, 'learning_rate': 1.5324326052059918e-05, 'batch_size': 64, 'epochs': 30, 'm2/activation': 'ReLU', 'hidden_size': 228}, user_a

In [ ]:
# Optuna의 큰 장점은 조건부 파라미터(Conditional Parameter)를 설정 할수 있는 것
# 분기 처리를 통해 Conditional 을 구성할 수 있음
# 어떤 옵티마이저를 샘플링 하냐에 따라서 다른 탐색 공간을 가져감

# 고정된 n,nc,rc

In [ ]:
def objective(trial: optuna.trial.Trial) -> float:
    """Optuna 최적화 목표 함수"""
    hyperparams = search_hyperparam(trial)  # trial을 올바르게 전달
    model = search_model(trial)  # trial을 올바르게 전달
    train_loader = DataLoader([(torch.rand(10), torch.randint(0, 2, (1,))) for _ in range(100)], batch_size=hyperparams["batch_size"], shuffle=True)
    trained_model = train_model(model, hyperparams, train_loader)
    score = evaluate(trained_model)
    return score

In [160]:
def objective(trial):
    model = []

    # Sample Normal Cell (NC)
    nc = trial.suggest_categorical("normal_cell", ["Conv", "DWConv", "Bottleneck", "InvertedResidualv2", "MBConv"])
    nc_args = []

    if nc == "Conv":
        out_channel = trial.suggest_int("normal_cell/out_channels", 16, 64)
        kernel_size = trial.suggest_int("normal_cell/kernel_size", 1, 7, step=2)
        activation = trial.suggest_categorical("normal_cell/activation", ["RELU", "RELU6", "Hardswish"])
        nc_args = [out_channel, kernel_size, 1, None, 1, activation]
    elif nc == "DWConv":
        # DWConv 파라미터 설정 추가
        pass
    elif nc == "Bottleneck":
        # Bottleneck 파라미터 설정 추가
        pass
    elif nc == "InvertedResidualv2":
        # InvertedResidualv2 파라미터 설정 추가
        pass
    elif nc == "MBConv":
        # MBConv 파라미터 설정 추가
        pass

    # Sample Reduction Cell (RC)
    rc_args = []
    rc = trial.suggest_categorical("reduction_cell", ["InvertedResidualv2", "InvertedResidualv3", "MaxPool", "AvgPool"])

    if rc == "InvertedResidualv2":
        t = trial.suggest_int("reduction_cell/t", 1, 6)
        out_channel = trial.suggest_int("reduction_cell/out_channels", 16, 64)
        stride = 2
        rc_args = [t, out_channel, stride]
    elif rc == "InvertedResidualv3":
        # InvertedResidualv3 파라미터 설정 추가
        pass
    elif rc == "MaxPool":
        # MaxPool 파라미터 설정 추가
        pass
    elif rc == "AvgPool":
        # AvgPool 파라미터 설정 추가
        pass

    # 모델 구성
    n = trial.suggest_int("n_repeat", 1, 3)  # 예시로 n 값을 설정
    model.append([n, nc, nc_args])  # n, nc, nc_args로 모델 레이어를 설정
    model.append([1, rc, rc_args])  # rc_args 설정
    model.append([n, nc, nc_args])  # n, nc, nc_args로 모델 레이어 설정
    model.append([1, rc, rc_args])  # rc_args 설정
    model.append([n, nc, nc_args])  # n, nc, nc_args로 모델 레이어 설정
    model.append([1, "GlobalAvgPool", []])  # GlobalAvgPool
    model.append([1, "Flatten", []])  # Flatten
    model.append([1, "Linear", [10]])  # Linear

    # 모델 구조를 반환 (혹은 이 모델 구조를 평가하여 성능을 반환)
    return len(model)  # 모델 크기를 반환 (예시로 사용)

In [161]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)

[I 2025-04-07 00:47:40,793] A new study created in memory with name: no-name-6f200e47-03e3-4524-a66e-835c3adb8308
[I 2025-04-07 00:47:40,796] Trial 0 finished with value: 8.0 and parameters: {'normal_cell': 'MBConv', 'reduction_cell': 'InvertedResidualv3', 'n_repeat': 2}. Best is trial 0 with value: 8.0.
[I 2025-04-07 00:47:40,801] Trial 1 finished with value: 8.0 and parameters: {'normal_cell': 'Conv', 'normal_cell/out_channels': 64, 'normal_cell/kernel_size': 1, 'normal_cell/activation': 'RELU6', 'reduction_cell': 'MaxPool', 'n_repeat': 3}. Best is trial 0 with value: 8.0.
[I 2025-04-07 00:47:40,804] Trial 2 finished with value: 8.0 and parameters: {'normal_cell': 'MBConv', 'reduction_cell': 'MaxPool', 'n_repeat': 1}. Best is trial 0 with value: 8.0.
[I 2025-04-07 00:47:40,806] Trial 3 finished with value: 8.0 and parameters: {'normal_cell': 'DWConv', 'reduction_cell': 'InvertedResidualv2', 'reduction_cell/t': 5, 'reduction_cell/out_channels': 51, 'n_repeat': 3}. Best is trial 0 with

# 고정된 nc,rc N은 점점 커지게

In [164]:
def objective(trial):
    model = []

    # Sample Normal Cell (NC)
    nc = trial.suggest_categorical("normal_cell", ["Conv", "DWConv", "Bottleneck", "InvertedResidualv2", "MBConv"])
    nc_args = []

    if nc == "Conv":
        out_channel = trial.suggest_int("normal_cell/out_channels", 16, 64)
        kernel_size = trial.suggest_int("normal_cell/kernel_size", 1, 7, step=2)
        activation = trial.suggest_categorical("normal_cell/activation", ["RELU", "RELU6", "Hardswish"])
        nc_args = [out_channel, kernel_size, 1, None, 1, activation]
    elif nc == "DWConv":
        # DWConv 파라미터 설정 추가
        pass
    elif nc == "Bottleneck":
        # Bottleneck 파라미터 설정 추가
        pass
    elif nc == "InvertedResidualv2":
        # InvertedResidualv2 파라미터 설정 추가
        pass
    elif nc == "MBConv":
        # MBConv 파라미터 설정 추가
        pass

    # Sample Reduction Cell (RC)
    rc_args = []
    rc = trial.suggest_categorical("reduction_cell", ["InvertedResidualv2", "InvertedResidualv3", "MaxPool", "AvgPool"])

    if rc == "InvertedResidualv2":
        t = trial.suggest_int("reduction_cell/t", 1, 6)
        out_channel = trial.suggest_int("reduction_cell/out_channels", 16, 64)
        stride = 2
        rc_args = [t, out_channel, stride]
    elif rc == "InvertedResidualv3":
        # InvertedResidualv3 파라미터 설정 추가
        pass
    elif rc == "MaxPool":
        # MaxPool 파라미터 설정 추가
        pass
    elif rc == "AvgPool":
        # AvgPool 파라미터 설정 추가
        pass

    # 모델 구성
    n = trial.suggest_int("n_repeat", 1, 3)  # 예시로 n 값을 설정
    n1, n2, n3 = n, 2*n, 3*n
    model.append([n1, nc, nc_args])  # n, nc, nc_args로 모델 레이어를 설정
    model.append([1, rc, rc_args])  # rc_args 설정
    model.append([n2, nc, nc_args])  # n, nc, nc_args로 모델 레이어 설정
    model.append([1, rc, rc_args])  # rc_args 설정
    model.append([n3, nc, nc_args])  # n, nc, nc_args로 모델 레이어 설정
    model.append([1, "GlobalAvgPool", []])  # GlobalAvgPool
    model.append([1, "Flatten", []])  # Flatten
    model.append([1, "Linear", [10]])  # Linear

    # 모델 구조를 반환 (혹은 이 모델 구조를 평가하여 성능을 반환)
    return len(model)  # 모델 크기를 반환 (예시로 사용)

In [165]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)

[I 2025-04-07 00:49:36,983] A new study created in memory with name: no-name-53b824e8-0629-42d6-971d-5face31edc9a
[I 2025-04-07 00:49:36,989] Trial 0 finished with value: 8.0 and parameters: {'normal_cell': 'MBConv', 'reduction_cell': 'InvertedResidualv2', 'reduction_cell/t': 6, 'reduction_cell/out_channels': 51, 'n_repeat': 1}. Best is trial 0 with value: 8.0.
[I 2025-04-07 00:49:36,991] Trial 1 finished with value: 8.0 and parameters: {'normal_cell': 'DWConv', 'reduction_cell': 'InvertedResidualv3', 'n_repeat': 3}. Best is trial 0 with value: 8.0.
[I 2025-04-07 00:49:36,994] Trial 2 finished with value: 8.0 and parameters: {'normal_cell': 'MBConv', 'reduction_cell': 'MaxPool', 'n_repeat': 3}. Best is trial 0 with value: 8.0.
[I 2025-04-07 00:49:36,997] Trial 3 finished with value: 8.0 and parameters: {'normal_cell': 'DWConv', 'reduction_cell': 'InvertedResidualv2', 'reduction_cell/t': 6, 'reduction_cell/out_channels': 44, 'n_repeat': 1}. Best is trial 0 with value: 8.0.
[I 2025-04-07

In [171]:
def objective(trial):
    model = []

    # Sample Normal Cell (NC)
    nc = trial.suggest_categorical("normal_cell", ["Conv", "DWConv", "Bottleneck", "InvertedResidualv2", "MBConv"])
    nc_args = []

    if nc == "Conv":
        out_channel = trial.suggest_int("normal_cell/out_channels", 16, 64)
        kernel_size = trial.suggest_int("normal_cell/kernel_size", 1, 7, step=2)
        activation = trial.suggest_categorical("normal_cell/activation", ["RELU", "RELU6", "Hardswish"])
        nc_args = [out_channel, kernel_size, 1, None, 1, activation]
    elif nc == "DWConv":
        # DWConv 파라미터 설정 추가
        pass
    elif nc == "Bottleneck":
        # Bottleneck 파라미터 설정 추가
        pass
    elif nc == "InvertedResidualv2":
        # InvertedResidualv2 파라미터 설정 추가
        pass
    elif nc == "MBConv":
        # MBConv 파라미터 설정 추가
        pass

    # Sample Reduction Cell (RC)
    rc_args = []
    rc = trial.suggest_categorical("reduction_cell", ["InvertedResidualv2", "InvertedResidualv3", "MaxPool", "AvgPool"])

    if rc == "InvertedResidualv2":
        t = trial.suggest_int("reduction_cell/t", 1, 6)
        out_channel = trial.suggest_int("reduction_cell/out_channels", 16, 64)
        stride = 2
        rc_args = [t, out_channel, stride]
    elif rc == "InvertedResidualv3":
        # InvertedResidualv3 파라미터 설정 추가
        pass
    elif rc == "MaxPool":
        # MaxPool 파라미터 설정 추가
        pass
    elif rc == "AvgPool":
        # AvgPool 파라미터 설정 추가
        pass

    # 모델 구성
    n = trial.suggest_int("n_repeat", 1, 3)  # 예시로 n 값을 설정
    n1 = trial.suggest_int("n_repeat", 1, 3)
    n2 = trial.suggest_int("n_repeat", 2, 5)
    n3 = trial.suggest_int("n_repeat", 3, 8)

    model.append([n1, nc, nc_args])  # n, nc, nc_args로 모델 레이어를 설정
    model.append([1, rc, rc_args])  # rc_args 설정
    model.append([n2, nc, nc_args])  # n, nc, nc_args로 모델 레이어 설정
    model.append([1, rc, rc_args])  # rc_args 설정
    model.append([n3, nc, nc_args])  # n, nc, nc_args로 모델 레이어 설정
    model.append([1, "GlobalAvgPool", []])  # GlobalAvgPool
    model.append([1, "Flatten", []])  # Flatten
    model.append([1, "Linear", [10]])  # Linear

    # 모델 구조를 반환 (혹은 이 모델 구조를 평가하여 성능을 반환)
    return len(model)  # 모델 크기를 반환 (예시로 사용)

In [172]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)

[I 2025-04-07 00:51:28,823] A new study created in memory with name: no-name-0821f094-9bdd-4040-acd7-48f68fdea09b
[I 2025-04-07 00:51:28,827] Trial 0 finished with value: 8.0 and parameters: {'normal_cell': 'DWConv', 'reduction_cell': 'AvgPool', 'n_repeat': 3}. Best is trial 0 with value: 8.0.
[I 2025-04-07 00:51:28,831] Trial 1 finished with value: 8.0 and parameters: {'normal_cell': 'Conv', 'normal_cell/out_channels': 31, 'normal_cell/kernel_size': 7, 'normal_cell/activation': 'Hardswish', 'reduction_cell': 'InvertedResidualv2', 'reduction_cell/t': 5, 'reduction_cell/out_channels': 49, 'n_repeat': 3}. Best is trial 0 with value: 8.0.
[I 2025-04-07 00:51:28,834] Trial 2 finished with value: 8.0 and parameters: {'normal_cell': 'MBConv', 'reduction_cell': 'MaxPool', 'n_repeat': 2}. Best is trial 0 with value: 8.0.
[I 2025-04-07 00:51:28,836] Trial 3 finished with value: 8.0 and parameters: {'normal_cell': 'Bottleneck', 'reduction_cell': 'AvgPool', 'n_repeat': 1}. Best is trial 0 with va

# ModuleGenerator의
# ymal에서 Model 만들기

In [173]:
class ModelParser:
	def __init__(self, cfg: str = ".model.yaml") -> None:
		with open(cfg) as f:
			self.cfg = yaml.load(f, Loader=yaml.FullLoader)

		self.in_channel = self.cfg["input_channel"],

		self.depth_multiply = self.cfg["depth_multiple"]
		self.width_multiply = self.cfg["width_multiple"]

		self.model_cfg = self.cfg["backbone"]
		self.model = self._parse_model()

In [175]:
def _parse_model(self) -> nn.Module:
	"""Parse model."""
  layers: List[nn.Module] = []
  in_channel = self.in_channel
	for i, (repeat, module, args) in enumerate(self.model_cfg):
		repeat = max(round(repeat * self.depth_multiply), 1) if repeat > 1 else repeat
		module_generator = ModuleGenerator(module)(
	    *args,
			width_multiply=self.width_multiply,
    )
    m = module_generator(repeat=repeat)

		layers.append(m)
    in_channel = module_generator.out_channel

	parsed_model = nn.Sequential(*layers)
  return parsed_model

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 3)

In [ ]:
# depth_multiple 과 width_multiple 이라는 파라미터는 MobileNetV1 에서 처음 등장한 컨셉 간략하게 설명드리자면
# 하나의 모델을 여러개의 Device 에서 쓰기 위해 유연함을 주는 개념
# depth_multiple 은 반복횟수를 조절
# width_multiple 은 채널의 크기를 조절
# 두 파라미터 depth_multiple 과 width_multiple 은 모델의 크기를 조율하는데 효과적인 파라미터

In [ ]:
class Bottleneck(nn.Module):
  """Simplified bottleneck block."""
  def __init__(
			self,
			in_channel: int,
			out_channels: int,
			shortcut=True,
			groups: int = 1,
			expansion: float = 0.5,
			activation: union[str, None] = "ReLU"
		) -> None:
    """Initialize."""
    super().__init__()
    expansion_channel = int(out_channels * expansion)

    self.conv1 = Conv(in_channel, expansion_channel, 1, 1, activation=activation)
    self.conv2 = Conv(expansion_channel, out_channels, 3, 1, groups=groups)
    self.shortcut = shortcut and in_channel == out_channels

  def forward(self, x: torch. Tensor) -> torch. Tensor:
    """Forward."""
    out = self.conv2( self.conv1(x))
    if self.shortcut:
    	out = out + x
    return out

NameError: name 'union' is not defined

In [ ]:
class BottleneckGenerator(GeneratorAbstract):
	"""Bottleneck block generator."""
	def __init__(self, *args, **kwargs):
		super().__init__(*args, **kwargs)

	@property
	def out_channel(self) -> int:
		"""Get out channel size."""
		return self._get_divisible_channel(self.args[0] * self.width_multiply)

	@property
	def base_module(self) -> nn.Module:
		"""Returns module class from src.modules based on the class name."""
		return getattr(__import__("src.modules", fromlist=[""]), self.name)

	def __call__(self, repeat: int = 1):
		repeat_args = [self.in_channel, self.in_channel, *self.args[1:]]
		args = [self.in_channel, self.out_channel, *self.args[1:]]
		module = []
		if repeat > 1:
			for _ in range(repeat-1):
				module.append(self.base_module(*repeat_args))
		module.append(self.base_module(*args))
		return self._get_module(module)

SyntaxError: expected ':' (<ipython-input-106-bd7e1be2ac38>, line 16)

In [ ]:
# 아웃풋 채널(self.args[0])에 width_multiply를 곱한 값이 out_channel의 값
# call 함수에서는 repeat 값을 입력받아서 주어진 반복 횟수만큼 모듈을 리스트에 추가하는 것을 확인